In [116]:
import pandas as pd, os, json, requests
import numpy as np, pickle, math, time
from itertools import combinations, permutations, product
from scipy import spatial
from scipy import stats
from tqdm import tqdm
from collections import Counter

In [2]:
# 设置显示宽度
np.set_printoptions(linewidth=400)

In [3]:
# 定义数据集路径
QWS_file_path = '../code/qws2.CSV'

In [4]:
# 读取数据
data = pd.read_table(QWS_file_path, header=None, delimiter=',')
data

,0,1,2,3,4,5,6,7,8,9,10
0,Response Time,Availability,Throughput,Successability,Reliability,Compliance,Best Practices,Latency\t,Documentation,Service Name ...,WSDL Address
1,302.75,89,7.1,90,73,78,80,187.75,32,MAPPMatching ...,http://xml.assessment.com/service/MAPPMatching...
2,482,85,16,95,73,100,84,1,2,Compound2 ...,http://www.mssoapinterop.org/asmx/WSDL/compoun...
3,3321.4,89,1.4,96,73,78,80,2.6,96,USDAData ...,http://www.strikeiron.com/webservices/usdadata...
4,126.17,98,12,100,67,78,82,22.77,89,GBNIRHolidayDates ...,http://www.holidaywebservice.com/Holidays/GBNI...
...,...,...,...,...,...,...,...,...,...,...,...
2503,200.8,93,2.4,98,73,100,84,7.4,41,garnierService ...,http://genome.dkfz-heidelberg.de/menu/hobit/em...
2504,56.17,97,11.3,97,83,78,91,7.17,3,AWSAlexa ...,http://awis.amazonaws.com/AWSAlexa/AWSAlexa.wsdl
2505,93.93,80,2.1,80,67,78,82,3.72,60,interop2 ...,http://www.cs.fsu.edu/~engelen/interop2.wsdl
2506,106.75,86,1.3,95,80,78,87,1.25,96,SailboatCalcsWS ...,http://pooh.poly.asu.edu/cst556-sailboatcalcsw...


In [5]:
# 获取数据，忽略标题行，和非功能属性
pd_data = data.iloc[1:, :8].values.astype(float)
item_size = pd_data.shape[0]
data_dimensions = pd_data.shape[1]
print(f'item_size: {item_size}\ndata_dimensions: {data_dimensions}')

item_size: 2507
data_dimensions: 8


In [6]:
# 每一列的最大值和最小值
column_max = np.max(pd_data, axis=0)
column_min = np.min(pd_data, axis=0)
# print(f'column_max: {column_max}\ncolumn_min: {column_min}')

# 正负属性，如延迟越高越不好，归一化
pos_or_neg = ['-', '+', '+', '+', '+', '+', '+', '-']
for (index, value) in enumerate(pos_or_neg):
    if value == '-':  # 如果是负属性，改变方向
        pd_data[:, index] = (pd_data[:, index] - column_min[index]) / (column_max[index] - column_min[index])
    else:
        pd_data[:, index] = (column_max[index] - pd_data[:, index]) / (column_max[index] - column_min[index])

all_services = pd_data
print(f'all_services: \n{all_services}\nall_services.shape: {all_services.shape}')
# 数据集的划分
constrains_index = np.random.choice(item_size, 6, replace=False)  # 随机选择6个索引
constrains = pd_data[constrains_index, :]  # 选择6个索引的数据，作为约束集
print(f"constrains_service: \n{constrains}\nconstrains_service.shape:{constrains.shape}")

candidates = np.delete(pd_data, constrains_index, axis=0)  # 删除6个索引的数据，作为候选集
print(f"candidates_service: \n{candidates}\ncandidates_service.shape:{candidates.shape}")

all_services: 
[[5.36579259e-02 1.18279570e-01 8.37209302e-01 ... 3.28358209e-01 3.33333333e-01 4.52887611e-02]
 [8.98505251e-02 1.61290323e-01 6.30232558e-01 ... 0.00000000e+00 2.44444444e-01 1.81155045e-04]
 [6.63157448e-01 1.18279570e-01 9.69767442e-01 ... 3.28358209e-01 3.33333333e-01 5.67619140e-04]
 ...
 [1.14948099e-02 2.15053763e-01 9.53488372e-01 ... 3.28358209e-01 2.88888889e-01 8.38144006e-04]
 [1.40833126e-02 1.50537634e-01 9.72093023e-01 ... 3.28358209e-01 1.77777778e-01 2.41540059e-04]
 [5.64342062e-02 3.01075269e-01 6.32558140e-01 ... 1.64179104e-01 1.77777778e-01 1.26808531e-03]]
all_services.shape: (2507, 8)
constrains_service: 
[[8.81746613e-03 6.66666667e-01 9.48837209e-01 6.73913043e-01 1.60714286e-01 1.64179104e-01 3.77777778e-01 8.26067003e-04]
 [9.36363618e-02 4.73118280e-01 8.93023256e-01 4.67391304e-01 2.85714286e-01 1.64179104e-01 4.44444444e-01 2.04101350e-02]
 [6.68326377e-02 1.07526882e-01 7.18604651e-01 3.26086957e-02 5.17857143e-01 3.28358209e-01 4.666666

In [7]:
# 生成历史调用记录，随机从所有服务中挑选出 10 - 15 个服务
gen_histories = lambda all_services: all_services[np.random.choice(all_services.shape[0], np.random.randint(10, 15 + 1), replace=False)]

In [8]:
# 生成很多个用户的历史调用记录
gen_users_histories = lambda user_count, all_services: np.array([gen_histories(all_services) for _ in range(user_count)], dtype=list)

In [9]:
users_histories =gen_users_histories(6, all_services)

In [10]:
# 获取用户历史记录的香农信息熵，表示多样化程度
def get_shannon_entropies(histories):
    indexs = [[np.argmax(i) for i in item] for item in histories]
    shannon_entropies = []
    for item in indexs:
        shannon_entropy = np.abs(sum([count / len(item) * (math.log2(count / len(item))) for count in Counter(item).values()]))
        shannon_entropies.append(shannon_entropy)
    return np.array(shannon_entropies)

In [11]:
# 归一化香农熵，获取用户的多样化 fu 参数; H0 表示超参数，需要调参
fus = lambda shannon_entropies, H0 : np.asarray([(item - np.min(shannon_entropies) + H0) / (np.max(shannon_entropies) - np.min(shannon_entropies) + H0) for item in shannon_entropies])
fus(get_shannon_entropies(users_histories), H0=1)

array([0.76321774, 1.        , 0.56394316, 0.95861474, 0.56394316, 0.56394316])

In [12]:
# 获取两个列表之间的相似度，综合相似度，alpha 是一个固定的参数，dimensions 表示 item 的维度
def get_similarity(item1, item2, alpha=0.5, dimensions=8):
    distance = spatial.distance.euclidean(item1, item2)
    tau = stats.kendalltau(item1, item2).correlation
    similarity = alpha * (1.0 - distance / np.sqrt(dimensions)) + (1.0 - alpha) * tau
    return similarity

In [13]:
# 根据约束服务和候选服务，生成推荐列表，fu 是个性参数，alpha是一个超参数
def get_kernel_matrix(constraints, candidates, fu, alpha, dimensions, alpha1):
    similarities  = np.asarray([get_similarity(item, constraints, dimensions=dimensions) for item in candidates])
    kernel_matrix = np.diag(np.square(similarities))
    comb = [(i,j) for (i, j) in list(combinations(range(len(candidates)), 2))]
    for (i, j) in tqdm(comb):
        kernel_matrix[i, j] = fu * alpha * similarities[i] * similarities[j] * get_similarity(candidates[i], candidates[j], alpha=alpha1,dimensions=dimensions)
        kernel_matrix[j, i] = kernel_matrix[i, j]
    return kernel_matrix

In [14]:
# dpp 核心算法，max_length 是 topK
def dpp(kernel_matrix, max_length, epsilon=1E-10):
    item_size = kernel_matrix.shape[0]
    cis = np.zeros((max_length, item_size))
    di2s = np.copy(np.diag(kernel_matrix))
    selected_items = list()
    selected_item = np.argmax(di2s)
    selected_items.append(selected_item)
    while len(selected_items) < max_length:
        k = len(selected_items) - 1
        ci_optimal = cis[:k, selected_item]
        di_optimal = math.sqrt(di2s[selected_item])
        elements = kernel_matrix[selected_item, :]
        eis = (elements - np.dot(ci_optimal, cis[:k, :])) / di_optimal
        cis[k, :] = eis
        di2s -= np.square(eis)
        di2s[selected_item] = -np.inf
        selected_item = np.argmax(di2s)
        if di2s[selected_item] < epsilon:
            break
        selected_items.append(selected_item)
    return selected_items

In [15]:
# 获取推荐列表与参考服务相比较精确度
def get_dcg_value(constraint, result_list, alpha, dimensions):
    gain = lambda score, rank: (np.power(2, score) - 1) / np.log2(1 + rank)
    dcg = np.sum([gain(get_similarity(item, constraint, alpha=alpha, dimensions=dimensions), index+1) for (index, item) in enumerate(result_list)])
    return dcg

In [75]:
# 获取某个列表的多样性，用累计不相似度来表示，alpha 是一个固定的参数，dimensions 表示 item 的维度
def get_diversity_of_list(hlist, alpha=0.5, dimensions=8):
    if len(hlist) <= 1:
        return 0
    return 2 / (len(hlist) * (len(hlist) - 1)) * np.sum([1 - get_similarity(hlist[i], hlist[j], alpha, dimensions=dimensions) for i, j in list(permutations(range(len(hlist)), 2))])

In [27]:
# 获取两个列表的多样性均方根误差，top_k 表示推荐的服务数量
def get_rmdse_of_lists(historical_list, recommend_list, top_k, dimensions):
    historical_diversity = get_diversity_of_list(historical_list, dimensions=dimensions)
    recommend_diversity = get_diversity_of_list(recommend_list, dimensions=dimensions)
    return np.sqrt(np.square((historical_diversity - recommend_diversity)) / (top_k))

In [18]:
# 控制变量法比较候选集数量，属性维度，推荐数量
n_list = [1000, 1300, 1600, 1900, 2200, 2500]
d_list = [3, 4, 5, 6, 7, 8]
top_k = [3, 4, 5, 6, 7, 8]

In [95]:
n_list = [100, 130, 160, 190, 220, 250]
d_list = [3, 4, 5, 6, 7, 8]
top_k = [3, 4, 5, 6, 7, 8]

In [96]:
# 使用 dpp 评判
dpp_res = {}

In [85]:
# dpp_res[f'1_2_3'] = 1, 2, 3

In [86]:
# dpp_res

{'1_2_3': (1, 2, 3)}

In [97]:
# 根据控制变量法，设置设置实验参数变化的影响结果
exp_list = list(product(n_list, d_list, top_k))

In [98]:
def dpp_eva(n, dimension, topK):
    _constraint = constrains[0, :dimension]
    _candidates = candidates[:n, :dimension]
    _kernel_matrix = get_kernel_matrix(_constraint, _candidates, fu=1, alpha=1, alpha1=0.5, dimensions=dimension)
    _result_list = dpp(_kernel_matrix, topK)
    _dcg = get_dcg_value(_constraint, _candidates[_result_list], alpha=1, dimensions=dimension)
    _diversity = get_diversity_of_list(_candidates[_result_list])
    _rmdse = get_rmdse_of_lists([_constraint], _candidates[_result_list], topK, dimensions=dimension)
    return _dcg, _diversity, _rmdse

In [99]:
dpp_eva(100, 3, 3)

100%|██████████| 4950/4950 [00:01<00:00, 3608.61it/s]


(1.4932928920713913, 0.7220132092643703, 0.5182944607728249)

In [119]:
def send_msg(message='空消息'):
    msg = {}
    msg['msg_type'] = 'text'
    msg['content'] = {'text': f'{message}\n{time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())}'}
    url = 'https://open.feishu.cn/open-apis/bot/v2/hook/36921b6c-7587-4781-8645-2d794b78bf3b'
    headers = {'Content-Type': 'application/json'}
    requests.post(url, json=msg, headers=headers)

In [122]:
t = time.time()
count = 0
for (n, d, topK) in tqdm(exp_list):
    count += 1
    if(count == 1):
        send_msg(f'DPP 开始, 参数n={n}, d={d}, topK={topK}')
    if count % 10 == 0:
        send_msg(f'DPP 计算中, {count}/{len(exp_list) - count}, 当前参数n={n}, d={d}, topK={topK}, 耗时{time.time() - t}')
    dcg, div, rmdse = dpp_eva(n, d, topK)
    dpp_res[f'{n}_{d}_{topK}'] = {'n': n, 'd': d, 'k': topK,'dcg': dcg, 'div': div, 'rmdse': rmdse}

100%|██████████| 216/216 [18:19<00:00,  5.09s/it]


In [123]:
dpp_res

{'100_3_3': {'n': 100,
  'd': 3,
  'k': 3,
  'dcg': 1.4932928920713913,
  'div': 0.7220132092643703,
  'rmdse': 0.5182944607728249},
 '100_3_4': {'n': 100,
  'd': 3,
  'k': 4,
  'dcg': 1.7394989535843577,
  'div': 0.5733346451007884,
  'rmdse': 0.3626269171429629},
 '100_3_5': {'n': 100,
  'd': 3,
  'k': 5,
  'dcg': 1.9895380464914987,
  'div': 0.48475122830810086,
  'rmdse': 0.2785234039395079},
 '100_3_6': {'n': 100,
  'd': 3,
  'k': 6,
  'dcg': 2.2856922224681377,
  'div': 0.44830965423524005,
  'rmdse': 0.24144679700064814},
 '100_3_7': {'n': 100,
  'd': 3,
  'k': 7,
  'dcg': 2.5187145323931888,
  'div': 0.39781542862010744,
  'rmdse': 0.19996578905091586},
 '100_3_8': {'n': 100,
  'd': 3,
  'k': 8,
  'dcg': 2.786148750745955,
  'div': 0.3620378032230707,
  'rmdse': 0.17172314338160197},
 '100_4_3': {'n': 100,
  'd': 4,
  'k': 3,
  'dcg': 1.6955316055800793,
  'div': 0.49560691856709893,
  'rmdse': 0.3515177296634733},
 '100_4_4': {'n': 100,
  'd': 4,
  'k': 4,
  'dcg': 1.976274834

In [124]:
dpp_res_json = json.dumps(dpp_res,indent=4)

In [125]:
dpp_res_json

'{\n    "100_3_3": {\n        "n": 100,\n        "d": 3,\n        "k": 3,\n        "dcg": 1.4932928920713913,\n        "div": 0.7220132092643703,\n        "rmdse": 0.5182944607728249\n    },\n    "100_3_4": {\n        "n": 100,\n        "d": 3,\n        "k": 4,\n        "dcg": 1.7394989535843577,\n        "div": 0.5733346451007884,\n        "rmdse": 0.3626269171429629\n    },\n    "100_3_5": {\n        "n": 100,\n        "d": 3,\n        "k": 5,\n        "dcg": 1.9895380464914987,\n        "div": 0.48475122830810086,\n        "rmdse": 0.2785234039395079\n    },\n    "100_3_6": {\n        "n": 100,\n        "d": 3,\n        "k": 6,\n        "dcg": 2.2856922224681377,\n        "div": 0.44830965423524005,\n        "rmdse": 0.24144679700064814\n    },\n    "100_3_7": {\n        "n": 100,\n        "d": 3,\n        "k": 7,\n        "dcg": 2.5187145323931888,\n        "div": 0.39781542862010744,\n        "rmdse": 0.19996578905091586\n    },\n    "100_3_8": {\n        "n": 100,\n        "d": 3

In [126]:
with open('dpp_res.json', 'w') as f:
    f.write(dpp_res_json)
    f.close()